In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
# download dataset
from urllib.request import urlretrieve
filename = os.path.join('..', 'data', 'datasets', 'kc_house_data.csv')
if not os.path.isfile(filename):
    print("Downloading %s" % filename)
    urlretrieve('https://s3.eu-central-1.amazonaws.com/ai-captain-public/datasets/kc_house_data.csv', filename) 

In [ ]:
data = pd.read_csv(filename)
data.shape

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data['pdate'] = pd.to_datetime(data.date)
data = data.set_index('pdate')

In [ ]:
data['price'].plot()
data['rprice'] = data['price'].rolling(10).mean()
data['rprice'].plot()

In [ ]:
data['bedrooms'].value_counts().plot(kind='bar')
plt.title('number of Bedroom')
plt.xlabel('Bedrooms')
plt.ylabel('Count')

In [ ]:
plt.scatter(data.price,data.sqft_living)
plt.title("Price vs Square Feet")

In [ ]:
plt.scatter(data.price,data.long)
plt.title("Price vs Location of the area")

In [ ]:
plt.scatter(data.price,data.lat)
plt.xlabel("Price")
plt.ylabel('Latitude')
plt.title("Latitude vs Price")

In [ ]:
plt.scatter(data.bedrooms,data.price)
plt.title("Bedroom and Price ")
plt.xlabel("Bedrooms")
plt.ylabel("Price")
plt.show()

In [ ]:
plt.scatter((data['sqft_living']+data['sqft_basement']),data['price'])

In [ ]:
plt.scatter(data.waterfront,data.price)
plt.title("Waterfront vs Price ( 0= no waterfront)")

In [ ]:
train1 = data.drop(['id', 'price'],axis=1)

In [ ]:
train1.head()

In [ ]:
data.floors.value_counts().plot(kind='bar')

In [ ]:
plt.scatter(data.floors,data.price)

In [ ]:
plt.scatter(data.condition,data.price)

In [ ]:
plt.scatter(data.zipcode,data.price)
plt.title("Which is the pricey location by zipcode?")

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression()

In [ ]:
data = data.dropna()
labels = data['price']
conv_dates = [1 if values == 2014 else 0 for values in data.date ]
data['date'] = conv_dates
train1 = data.drop(['id', 'price'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train1, labels, test_size = 0.10, random_state = 2)

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_test, y_test)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

model = make_pipeline(Normalizer(), LinearRegression())
model.fit(x_train,y_train)
model.score(x_test, y_test)

In [ ]:
plt.scatter(y_test, reg.predict(x_test))

In [ ]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
plt.scatter(y_test, clf.predict(x_test))

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    "learning_rate": [0.1, 0.01],
    "n_estimators": [100, 200]
}

clf = GridSearchCV(ensemble.GradientBoostingRegressor(), parameters, cv=3, n_jobs=-1)

clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))
print(clf.best_params_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    "learning_rate": np.linspace(0.1, 0.001, 10),
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth": [3,5,8],
    "max_features": ["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample": [0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators": [10, 20, 50, 100, 200]
}

clf = RandomizedSearchCV(ensemble.GradientBoostingRegressor(), parameters, cv=3, n_jobs=-1, n_iter=10)

clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))
print(clf.best_params_)